# Inserting and Updating Objects
Demonstrate creation and updating of different objects with the Rhino Health Python SDK

#### Prerequisites 
1. Have projects called "Test" and "Test2" that you have created
2. Have a data schema that you can use for importing a cohort
3. Have files on your Rhino Client that can be used to import a cohort with the data schema
4. Have a container image pushed to your ECR repo with any code (just used for creating an object, not for running the code)

### Initialization and Login

In [ ]:
import getpass
import rhino_health as rh
from rhino_health.lib.endpoints.endpoint import NameFilterMode, VersionMode
from rhino_health.lib.endpoints.aimodel.aimodel_dataclass import AIModelCreateInput
from rhino_health.lib.endpoints.cohort.cohort_dataclass import CohortCreateInput

In [ ]:
print("Logging In")
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health
my_workgroup_ecr_repo = "rhino-gc-workgroup-XXXXXXXXXXXXXX"  # Replace this with your workgroup's ECR repo
my_image_name = "upserting-objects"  # The name (tag) of the container image you pushed to your ECR repo
ecr_base_uri = rh.lib.constants.ECRService.PROD_URL
session = rh.login(username=my_username, password=getpass.getpass())
print("Logged In")

# Get Project By Name

In [ ]:
projects = session.project.search_for_projects_by_name("Test")
print("projects")
print([(x.name, x.uid) for x in projects][:10])

In [ ]:
project = session.project.get_project_by_name("Test2")
print(project)

In [ ]:
print(project.dict(include={'uid', 'name', 'description'}))

# Create Cohort

In [ ]:
data_schema_uid = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Replace data schema UID
cohort_csv_location = "/rhino_data/test/cohort_data.csv"  # Replace cohort CSV location

In [ ]:
cohort_input = CohortCreateInput(
            name="Upsert Cohort",
            description="Test Upsert Cohort",
            project_uid=project.uid,
            workgroup_uid=project.primary_workgroup_uid,
            data_schema_uid=data_schema_uid,
            csv_filesystem_location=cohort_csv_location,
            method="filesystem",
            is_data_deidentified=True,
        )
cohort = session.cohort.add_cohort(cohort_input)
print(cohort.uid)

In [ ]:
duplicate_cohort_input = session.cohort.add_cohort(cohort_input)
print(duplicate_cohort_input.uid)

In [ ]:
new_version_of_cohort = session.cohort.add_cohort(cohort_input, return_existing=False, add_version_if_exists=True)
print(new_version_of_cohort.uid, new_version_of_cohort.version)

# Get Cohort

In [ ]:
found_cohort = project.get_cohort_by_name("Upsert Cohort")
print(found_cohort.dict(include={'name', 'uid', 'version'}))

In [ ]:
older_cohort = project.get_cohort_by_name("Upsert Cohort", version=1)
print(older_cohort.dict(include={'name', 'uid', 'version'}))

In [ ]:
all_cohort_versions = project.search_for_cohorts_by_name("upsert", version=VersionMode.ALL, name_filter_mode=NameFilterMode.CONTAINS)
print([cohort.dict(include={'name', 'uid', 'version'}) for cohort in all_cohort_versions])

In [ ]:
non_existent_cohort = project.get_cohort_by_name("I do not exist")
print (non_existent_cohort)

# Create AIModel

In [ ]:
image_uri = f"{ecr_base_uri}/{my_workgroup_ecr_repo}:{my_image_name}"

test_model_object = AIModelCreateInput(
    name="Upsert Model",
    description="Test",
    input_data_schema_uids=[cohort.uid],
    output_data_schema_uids=[cohort.uid],
    project_uid=project.uid,
    model_type="Generalized Compute",
    config={"container_image_uri": image_uri},
)

aimodel = session.aimodel.create_aimodel(
    test_model_object
)

print(aimodel.uid)

In [ ]:
duplicate_aimodel_input = session.aimodel.create_aimodel(test_model_object)
print(duplicate_aimodel_input.uid)

In [ ]:
new_version_of_aimodel = session.aimodel.create_aimodel(test_model_object, return_existing=False, add_version_if_exists=True)
print(new_version_of_aimodel.dict(include={'name', 'uid', 'version'}))

# Get AIModel

In [ ]:
found_aimodel = project.get_aimodel_by_name(aimodel.name)
print(found_aimodel.dict(include={'name', 'uid', 'version'}))

In [ ]:
older_aimodel = project.get_aimodel_by_name(aimodel.name, version=0)
print(older_aimodel.dict(include={'name', 'uid', 'version'}))

In [ ]:
all_aimodel_versions = project.search_for_aimodels_by_name("upsert", version=VersionMode.ALL, name_filter_mode=NameFilterMode.CONTAINS)
print([aimodel.dict(include={'name', 'uid', 'version'}) for aimodel in all_aimodel_versions])

In [ ]:
non_existent_aimodel = project.get_aimodel_by_name("I do not exist")
print (non_existent_aimodel)

# Global Search

In [ ]:
partial_search = session.aimodel.search_for_aimodels_by_name(
    "Test", version=1, name_filter_mode=NameFilterMode.CONTAINS
)
print("search 1")
print([(x.name, x.version) for x in partial_search][:10])

In [ ]:
partial_search = session.aimodel.search_for_aimodels_by_name(
    "Upsert",
    version=VersionMode.LATEST,
    name_filter_mode=NameFilterMode.CONTAINS,
)
print("search latest")
print([(x.name, x.version) for x in partial_search])

In [ ]:
partial_search = session.aimodel.search_for_aimodels_by_name(
    "Upsert",
    version=VersionMode.ALL,
    name_filter_mode=NameFilterMode.CONTAINS,
)
print("search latest")
print([(x.name, x.version) for x in partial_search])